In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv("../data/DMS/antibody/_clustering.csv", index_col=0)
dms_scores = pd.read_csv("../data/DMS/antibody/_XBB15_JN1_aggregate_dms_scores.csv")
dms_scores_XBB = pd.read_csv("../data/DMS/antibody/dms_antibodies_XBB15_JN1_agg.csv").query("antigen == 'XBB.1.5_RBD'")
dms_scores_JN1 = pd.read_csv("../data/DMS/antibody/dms_antibodies_XBB15_JN1_agg.csv").query("antigen == 'JN.1_RBD'")


In [2]:
aas = ["A", "R", "N", "D", "C", "E", "Q", "G", "H", "I", "L", "K", "M", "F", "P", "S", "T", "W", "Y", "V"]
def refine_bindexpr(df, mutations = []):
    sites = range(331, 532)
    
    df = df[['site', 'mutation', 'expr_avg', 'bind_avg']]
    df = df.assign(site_mut = lambda x: x['site'].astype(str)+x['mutation'], bias_e=0.0, bias_b=0.0).reset_index(drop=True)
    
    mutations = [(int(x[0:-1]), x[-1]) for x in mutations]
    
    exist = set(df['site_mut'].to_list())
    for site in sites:
        for aa in aas:
            if str(site)+aa not in exist:
                df.loc[len(df)] = ({'site':site, 'mutation':aa, 'bind_avg': -100, 'expr_avg': -100, 'site_mut':str(site)+aa})
    for site, mut in mutations:
        expr = df.query('site == @site and mutation == @mut')['expr_avg'].item()
        bind = df.query('site == @site and mutation == @mut')['bind_avg'].item()
        df.loc[df['site'] == site, 'bias_e'] += expr
        df.loc[df['site'] == site, 'bias_b'] += bind
    
    print(df['bias_e'].abs().sum())
    df['expr_avg'] -= df['bias_e']
    df['bind_avg'] -= df['bias_b']
    
    return df.drop(columns=['bias_e','bias_b']).sort_values(['site', 'mutation']).fillna(-100)

In [3]:
# def nanmean(x,y):
#     assert(len(x) == len(y))
#     return [np.nanmean([x[i], y[i]]) for i in range(len(x))]

# libs = pd.read_csv("/dshare/xielab/analysisdata/Antibody/wangjing/yeast_display/pipeline/pipeline/libinfo.csv")
# df = pd.read_csv(
#     libs.query('library == "2mutJN1merged"')['bindexpr'].iloc[0])

# df2 = pd.read_csv("/dshare/xielab/analysisdata/Antibody/wangjing/yeast_display/Titeseq/2mutJN1lib3-20240419/single_mut_model.csv", index_col=0).merge(
#     pd.read_csv("/dshare/xielab/analysisdata/Antibody/wangjing/yeast_display/Titeseq/2mutJN1lib4-20240419/single_mut_model.csv",index_col=0),
#     on=['site', 'mutant'], how='outer'
# ).assign(bind_avg = lambda x: [np.nanmean([x['func_score_x'][i], x['func_score_y'][i]]) for i in range(len(x))]).rename(columns={'mutant':'mutation'})
# bindexpr_refined = refine_bindexpr(df.drop(columns='bind_avg').merge(df2[['site','mutation', 'bind_avg']], on=['site','mutation'], how='outer'), True)

XBB15_Tyler = pd.read_csv("../data/DMS/TStarrLab_RBD_DMS_scores.csv").query(
    'target == "Omicron_XBB15"')[['position','mutant','delta_expr','delta_bind']].rename(
        columns={
            "mutant":"mutation", "delta_bind":"bind_avg", "position":"site", "delta_expr": "expr_avg"
        })

bindexpr_refined_JN1 = refine_bindexpr(
    pd.read_csv("../data/DMS/RBD_expression/_JN1_DMS_single_mutation_expr.csv").merge(
    XBB15_Tyler[['site', 'mutation', 'bind_avg']], on=['site','mutation'], how='left'), 
    mutations = ['332V', '346R', '356T', '368L', '403K', '445H', '450D', '452W', '455S', '481K', '484K', '490F']
).query('mutation in @aas') # use XBB.1.5 binding data to simulate JN.1

# bindexpr_refined_JN1 = refine_bindexpr(
#     pd.read_csv("../data/DMS/RBD_expression/_JN1_DMS_single_mutation_expr.csv").merge(
#     pd.read_csv("../data/DMS/ACE2_binding/_JN1_DMS_single_mutation_bind.csv")[['site', 'mutation', 'bind_avg']], on=['site','mutation'], how='left'), 
#     mutations = []
# ).query('mutation in @aas') # use JN.1


bindexpr_refined_XBB = refine_bindexpr(XBB15_Tyler)

# bindexpr_refined = refine_bindexpr(pd.read_csv("../data/DMS/RBD_expression/_JN1_DMS_single_mutation_expr.csv").assign(bind_avg = 0.0), True) # only include expression
bindexpr_refined_JN1.query('site == 440')

0.0
0.0


,site,mutation,expr_avg,bind_avg,site_mut
2290,440,A,0.000000,-0.38,440A
2291,440,C,-0.219750,-0.69,440C
2292,440,D,-0.153421,-0.55,440D
2293,440,E,0.130723,-0.68,440E
2294,440,F,-0.569896,-0.55,440F
2295,440,G,-0.120437,-0.35,440G
2296,440,H,-0.103910,-0.29,440H
2297,440,I,-0.650035,-0.50,440I
2298,440,K,0.000000,0.00,440K
2299,440,L,-0.205306,-0.53,440L


In [4]:
reinfection_sources = ["BA.5 BTI + XBB infection", "BA.5 BTI + HK.3 infection", "BA.5 BTI + JN.1 infection"]
targets = {
    # "XBB_BTI": {
    #     "dms": dms_scores,
    #     'use_abs': data.query("source == 'XBB BTI'").index.to_list(),
    #     "neut": "JN1_IC50",
    #     # "codon_weights": "/gshare/xielab/jianfc/COVID/temp_tasks/calculator_codon/mutation_weights_XBB_1_5_EPI_ISL_17054053.csv",
    #     "codon_mut1": "../data/mut1_JN_1_EPI_ISL_18373905.csv",
    #     "bindexpr":bindexpr_refined,
    # },
    # "XBB": {
    #     "dms": dms_scores,
    #     'use_abs': data.query("source == 'XBB infection'").index.to_list(),
    #     "neut": "JN1_IC50",
    #     # "codon_weights": "/gshare/xielab/jianfc/COVID/temp_tasks/calculator_codon/mutation_weights_XBB_1_5_EPI_ISL_17054053.csv",
    #     "codon_mut1": "../data/mut1_JN_1_EPI_ISL_18373905.csv",
    #     "bindexpr":bindexpr_refined,
    # },
    # "BA5_XBB": {
    #     "dms": dms_scores,
    #     'use_abs': data.query("source == 'BA.5 + XBB infection'").index.to_list(),
    #     "neut": "JN1_IC50",
    #     # "codon_weights": "/gshare/xielab/jianfc/COVID/temp_tasks/calculator_codon/mutation_weights_XBB_1_5_EPI_ISL_17054053.csv",
    #     "codon_mut1": "../data/mut1_JN_1_EPI_ISL_18373905.csv",
    #     "bindexpr":bindexpr_refined,
    # },
    # "BA5_JN1": {
    #     "dms": dms_scores,
    #     'use_abs': data.query("source == 'BA.5 + JN.1 infection'").index.to_list(),
    #     "neut": "JN1_IC50",
    #     # "codon_weights": "/gshare/xielab/jianfc/COVID/temp_tasks/calculator_codon/mutation_weights_JN_1_EPI_ISL_18373905.csv",
    #     "codon_mut1": "../data/mut1_JN_1_EPI_ISL_18373905.csv",
    #     "bindexpr":bindexpr_refined,
    # },
    # "BA5_BTI_JN1": {
    #     "dms": dms_scores,
    #     'use_abs': data.query("source == 'BA.5 BTI + JN.1 infection'").index.to_list(),
    #     "neut": "JN1_IC50",
    #     # "codon_weights": "/gshare/xielab/jianfc/COVID/temp_tasks/calculator_codon/mutation_weights_JN_1_EPI_ISL_18373905.csv",
    #     "codon_mut1": "../data/mut1_JN_1_EPI_ISL_18373905.csv",
    #     "bindexpr":bindexpr_refined,
    # },
    # "BA5_BTI_HK3": {
    #     "dms": dms_scores,
    #     'use_abs': data.query("source == 'BA.5 BTI + HK.3 infection'").index.to_list(),
    #     "neut": "JN1_IC50",
    #     # "codon_weights": "/gshare/xielab/jianfc/COVID/temp_tasks/calculator_codon/mutation_weights_JN_1_EPI_ISL_18373905.csv",
    #     "codon_mut1": "../data/mut1_JN_1_EPI_ISL_18373905.csv",
    #     "bindexpr":bindexpr_refined,
    # },
    # "BA5_BTI_XBB": {
    #     "dms": dms_scores,
    #     'use_abs': data.query("source == 'BA.5 BTI + XBB infection'").index.to_list(),
    #     "neut": "JN1_IC50",
    #     # "codon_weights": "/gshare/xielab/jianfc/COVID/temp_tasks/calculator_codon/mutation_weights_JN_1_EPI_ISL_18373905.csv",
    #     "codon_mut1": "../data/mut1_JN_1_EPI_ISL_18373905.csv",
    #     "bindexpr":bindexpr_refined,
    # },
    # "XBB_cross": {
    #     "dms": dms_scores_XBB,
    #     'use_abs': data.query("source == 'BA.5 BTI + XBB infection' and paper_reactivity == 'cross'").index.to_list(),
    #     "neut": "XBB1_5_IC50",
    #     # "codon_weights": "/gshare/xielab/jianfc/COVID/temp_tasks/calculator_codon/mutation_weights_JN_1_EPI_ISL_18373905.csv",
    #     "codon_mut1": "../data/mut1_XBB_1_5_EPI_ISL_17054053.csv",
    #     "bindexpr":bindexpr_refined_XBB,
    # },
    # "XBB_specific": {
    #     "dms": dms_scores_XBB,
    #     'use_abs': data.query("source == 'BA.5 BTI + XBB infection' and paper_reactivity == 'specific'").index.to_list(),
    #     "neut": "XBB1_5_IC50",
    #     # "codon_weights": "/gshare/xielab/jianfc/COVID/temp_tasks/calculator_codon/mutation_weights_JN_1_EPI_ISL_18373905.csv",
    #     "codon_mut1": "../data/mut1_XBB_1_5_EPI_ISL_17054053.csv",
    #     "bindexpr":bindexpr_refined_XBB,
    # },
    # "JN1_cross": {
    #     "dms": dms_scores_JN1,
    #     'use_abs': data.query("source == 'BA.5 BTI + JN.1 infection' and paper_reactivity == 'cross'").index.to_list(),
    #     "neut": "JN1_IC50",
    #     # "codon_weights": "/gshare/xielab/jianfc/COVID/temp_tasks/calculator_codon/mutation_weights_JN_1_EPI_ISL_18373905.csv",
    #     "codon_mut1": "../data/mut1_JN_1_EPI_ISL_18373905.csv",
    #     "bindexpr":bindexpr_refined_JN1,
    # },
    # "JN1_specific": {
    #     "dms": dms_scores_JN1,
    #     'use_abs': data.query("source == 'BA.5 BTI + JN.1 infection' and paper_reactivity == 'specific'").index.to_list(),
    #     "neut": "JN1_IC50",
    #     # "codon_weights": "/gshare/xielab/jianfc/COVID/temp_tasks/calculator_codon/mutation_weights_JN_1_EPI_ISL_18373905.csv",
    #     "codon_mut1": "../data/mut1_JN_1_EPI_ISL_18373905.csv",
    #     "bindexpr":bindexpr_refined_JN1,
    # },
    "reinfection_cross_JN1": {
        "dms": dms_scores,
        'use_abs': data.query("source in @reinfection_sources and paper_reactivity == 'cross'").index.to_list(),
        "neut": "JN1_IC50",
        # "codon_weights": "/gshare/xielab/jianfc/COVID/temp_tasks/calculator_codon/mutation_weights_JN_1_EPI_ISL_18373905.csv",
        "codon_mut1": "../data/mut1_JN_1_EPI_ISL_18373905.csv",
        "bindexpr":bindexpr_refined_JN1,
    },
    "reinfection_cross_KP2": {
        "dms": dms_scores,
        'use_abs': data.query("source in @reinfection_sources and paper_reactivity == 'cross'").index.to_list(),
        "neut": "JN1_R346T_F456L_IC50",
        # "codon_weights": "/gshare/xielab/jianfc/COVID/temp_tasks/calculator_codon/mutation_weights_JN_1_EPI_ISL_18373905.csv",
        "codon_mut1": "../data/mut1_KP_2_EPI_ISL_18916251_del.csv",
        "bindexpr":bindexpr_refined_JN1,
        "mutations": ['346T', '456L']
    },
    "reinfection_cross_KP3": {
        "dms": dms_scores,
        'use_abs': data.query("source in @reinfection_sources and paper_reactivity == 'cross'").index.to_list(),
        "neut": "KP3_IC50",
        # "codon_weights": "/gshare/xielab/jianfc/COVID/temp_tasks/calculator_codon/mutation_weights_JN_1_EPI_ISL_18373905.csv",
        "codon_mut1": "../data/mut1_KP_3_EPI_ISL_19036766.csv",
        "bindexpr":bindexpr_refined_JN1,
        "mutations": ['456L', '493E']
    },
    "reinfection_specific_JN1": {
        "dms": dms_scores,
        'use_abs': data.query("source in @reinfection_sources and paper_reactivity == 'specific'").index.to_list(),
        "neut": "JN1_IC50",
        # "codon_weights": "/gshare/xielab/jianfc/COVID/temp_tasks/calculator_codon/mutation_weights_JN_1_EPI_ISL_18373905.csv",
        "codon_mut1": "../data/mut1_JN_1_EPI_ISL_18373905.csv",
        "bindexpr":bindexpr_refined_JN1,
    },
    "reinfection_specific_KP2": {
        "dms": dms_scores,
        'use_abs': data.query("source in @reinfection_sources and paper_reactivity == 'specific'").index.to_list(),
        "neut": "JN1_R346T_F456L_IC50",
        # "codon_weights": "/gshare/xielab/jianfc/COVID/temp_tasks/calculator_codon/mutation_weights_JN_1_EPI_ISL_18373905.csv",
        "codon_mut1": "../data/mut1_KP_2_EPI_ISL_18916251_del.csv",
        "bindexpr":bindexpr_refined_JN1,
        "mutations": ['346T', '456L']
    },
    "reinfection_specific_KP3": {
        "dms": dms_scores,
        'use_abs': data.query("source in @reinfection_sources and paper_reactivity == 'specific'").index.to_list(),
        "neut": "KP3_IC50",
        # "codon_weights": "/gshare/xielab/jianfc/COVID/temp_tasks/calculator_codon/mutation_weights_JN_1_EPI_ISL_18373905.csv",
        "codon_mut1": "../data/mut1_KP_3_EPI_ISL_19036766.csv",
        "bindexpr":bindexpr_refined_JN1,
        "mutations": ['456L', '493E']
    },
}

for target, d in targets.items():
    if "mutations" in d:
        d['bindexpr'] = refine_bindexpr(d['bindexpr'], mutations=d['mutations'])

7.659544260077392
6.39039844824715
7.659544260077392
6.39039844824715


In [5]:
def ic50_weighting(x, low=0.001, high=1):
    if pd.isna(x) or x > high:
        return 0
    if x < low:
        return 1
    res = -np.log10(x)
    res_l = -np.log10(low)
    res_h = -np.log10(high)
    return 1.0-(res-res_l)/(res_h-res_l)


In [6]:
import logomaker
from matplotlib import rcParams
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.font_manager

matplotlib.font_manager._load_fontmanager(try_read_cache=False)

rcParams['pdf.fonttype'] = 42
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['Arial']

def plot_res_logo(res, prefix, shownames={}, rownames=None, site_thres=0.0, force_plot_sites = None, force_ylim = None, width=None):
    flat_res = res.pivot(index=['antibody', 'site'], columns='mutation', values='mut_escape').fillna(0)
    sites_mean_score = flat_res.mean(axis=1)
    sites_total_score = flat_res.sum(axis=1)
    _ = sites_total_score[sites_total_score>site_thres].index
    strong_sites = np.unique(np.array(sorted([i[1] for i in _])))
    print(strong_sites)

    plot_sites = strong_sites
    plot_sites = plot_sites[plot_sites < 520].astype(int)
    print(plot_sites)
    
    if force_plot_sites is not None:
        plot_sites = force_plot_sites
    
    flat_res = flat_res[flat_res.index.isin(plot_sites, level=1)]

    _ = pd.DataFrame(sites_total_score)
    _.columns = ['value']
    _['site'] = [i[1] for i in _.index]
    _['antibody'] = [i[0] for i in _.index]

    if rownames is not None:
        Abs = rownames
    else:
        Abs = np.unique([i[0] for i in flat_res.index])
    print(Abs)
    Npages = len(Abs)//10 + 1
    if width is None:
        width=30
    with PdfPages(prefix+'_aa_logo.pdf') as pdf:
        for p in range(Npages):
            Abs_p = Abs[p*10:min(len(Abs),(p+1)*10)]
            fig = plt.figure(figsize=(width,len(Abs_p)*4.6)).subplots_adjust(wspace=0.2,hspace=0.5)
            site2pos = {}
            for i in range(len(plot_sites)):
                site2pos[plot_sites[i]] = i

            for i in range(len(Abs_p)):
                ab = Abs_p[i]
                _ = flat_res.query('antibody == @ab').droplevel(0)
                add_sites = np.setdiff1d(plot_sites, _.index)
                for _site in add_sites:
                    _.loc[_site,:] = 0.0
                _ = _.sort_index()
                _.index = range(len(_))
                ax = plt.subplot(len(Abs_p), 1, i+1)
                logo = logomaker.Logo(_,
                               ax=ax, 
                               color_scheme='dmslogo_funcgroup', 
                               vpad=.1, 
                               width=.8)
                logo.style_xticks(anchor=1, spacing=1, rotation=90, fontsize=16)
                _max = np.sum(_.to_numpy(), axis=1).max()
                # ax.set_xticklabels(plot_sites[1::2])
                ax.set_xticklabels(plot_sites)
                
                # ax.set_yticks([])
                ax.tick_params(axis='both', which='both', length=0)
                if force_ylim is not None:
                    ax.set_ylim(0.0,force_ylim)
                ax.yaxis.set_tick_params(labelsize=20)
                if ab in shownames:
                    ax.set_title(shownames[ab], fontsize=8, fontweight="bold")
                else:
                    ax.set_title(ab, fontsize=8, fontweight="bold")
            pdf.savefig()
            plt.close()

def do_calc(scores_r, ag, neut, A_adv = True, A_codon = True, A_neut = True,
            E=1.0, B=1.0, use_max=False, use_norm=False,
            logo=False, use_codon_weights=False, specific_weight = None,
            muts = [], title=None,blacklist=None
           ):
    if A_neut:
        neut_data = data[neut].to_dict()
    
    mut_expand = set()
    XTag = 0
    for _site, _m in muts:
        if _m == "X":
            XTag = 1
        mut_expand.add(str(_site)+_m)
    # print("out mut:",mut_expand)
    
    if A_adv:
        single_mut_effects = targets[ag]['bindexpr'].assign(
            coef=lambda x: [y for y in (np.tanh(x['expr_avg']*(x['expr_avg']<0)*E)+1)*(np.tanh(x['bind_avg']*B)+1)]
        ).set_index('site_mut')['coef'].to_dict()

    if A_codon:
        if use_codon_weights is False:
            use_codon = pd.read_csv(targets[ag]['codon_mut1'])
            _umuts = {}
            for i in range(len(use_codon)):
                _ms = use_codon['mut1'][i]
                for x in _ms:
                    _umuts[str(use_codon['pos'][i])+x] = 1.0
        else:
            _umuts = pd.read_csv(targets[ag]['codon_weights']).assign(
                mutation = lambda x: x['pos'].astype(str)+x['mut']).set_index('mutation')['weight'].to_dict()
    
    src_dict = data['source'].to_dict()
    
    if specific_weight is not None:
        spc_dict = data['paper_reactivity'].to_dict()
    
    scores = scores_r.assign(site_mut = lambda x: x['site'].astype(str)+x['mutation']).assign(
        adv_weight = (lambda x: [single_mut_effects[y] if (y in single_mut_effects and not np.isnan(single_mut_effects[y])) else 0.0 for y in x['site_mut']]) if A_adv else 1.0,
        codon_weight = (lambda x: [(_umuts[y] if y in _umuts else 0.0) for y in x['site_mut'].to_list()]) if A_codon else 1.0
    )
    
    if use_norm:
        scores = scores.assign(escape_max = lambda x: x.groupby('antibody')['mut_escape'].transform('max')).assign(
            mut_escape = lambda x: x['mut_escape']/x['escape_max']).drop(columns=['escape_max'])
    if XTag:
        _scX = scores.groupby(['antibody','site']).max().reset_index().assign(
            mutation = 'X',
            site_mut = lambda x: x['site'].astype(str)+'X'
        )
        scores = pd.concat([scores, _scX])
    
    
    _ab_mut_coef = {} if len(muts) == 0 else (
        scores.query('site_mut in @mut_expand')
              .groupby('antibody')['mut_escape']
              .agg(coef=lambda x: mut_agg_func(x, len(mut_expand)))
    )['coef'].to_dict()
    scores = scores.assign(mut_weight = lambda x: [(_ab_mut_coef[y] if y in _ab_mut_coef else 1.0) for y in x['antibody']]).query('mutation != "X"')
    
    scores = scores.assign(neut_weight = lambda x: [(0.0 if np.isnan(neut_data[y]) else ic50_weighting(neut_data[y])) if A_neut else 1.0 for y in x['antibody']])
    
    scores['other_weight'] = 1.0

    if specific_weight is not None:
        scores = scores.assign(other_weight = lambda x: [specific_weight[src_dict[y]] if (src_dict[y] in specific_weight and "specific" in spc_dict[y]) else 1.0 for y in x['antibody']])
    
    scores = scores.assign(
        mut_escape_adj = lambda x: x['mut_escape'] * x['neut_weight'] * x['adv_weight'] * x['codon_weight'] * x['other_weight'] * x['mut_weight']
    )
    _title = (
              'weight: '+ag+' expr_bind:'+str(A_adv)+
              ' codon:'+str(A_codon)+
              ' norm:'+str(use_norm)+' max:'+str(use_max)+
              ' Expr:'+str(E)+' Bind:'+str(B)) if title is None else title
    
    if blacklist is not None:
        scores = scores.assign(site_mut = lambda x: x['site'].astype(str)+x['mutation']).query('site_mut not in @blacklist')
    
    print(len(pd.unique(scores.query('mut_escape_adj > 0')['antibody'])))
    
    if logo:
        scores = scores.groupby(['site','mutation']).sum()['mut_escape_adj'].reset_index().assign(antibody=_title)
        scores['mut_escape_adj'] = scores['mut_escape_adj']/scores['mut_escape_adj'].max()
        return scores
    
    if use_max:
        site_avg = scores.groupby(['site', 'antibody'])['mut_escape_adj'].max().reset_index().groupby('site')['mut_escape_adj'].sum().reset_index()
    else:
        site_avg = scores.groupby(['site', 'antibody'])['mut_escape_adj'].sum().reset_index().groupby('site')['mut_escape_adj'].sum().reset_index()
   
    site_avg['mut_escape_adj'] = site_avg['mut_escape_adj']/site_avg['mut_escape_adj'].max()

    return site_avg.assign(
        group = ag, 
        weight = neut, is_expr_bind = A_adv, is_codon = A_codon, is_norm = use_norm, is_max = use_max, expr_coef = E, bind_coef = B
    ).rename(columns={'mut_escape_adj':'mut_escape'})


In [7]:
df = []
df_site = []

for ag in targets:
    use_abs = targets[ag]['use_abs']
    scores_orig = targets[ag]['dms'].query('antibody in @use_abs')

    df.append(do_calc(scores_orig, ag, neut=targets[ag]['neut'], A_adv = True, A_codon = True, A_neut=True, 
                      E=1.0, B=1.0, 
             use_norm=True, use_max=False, use_codon_weights=False, logo=True, muts=[]).rename(
        columns={'mut_escape_adj':'mut_escape'}).assign(
        target=ag,
        antibody=ag+' weighting '+targets[ag]['neut'],
    ))

    df_site.append(do_calc(scores_orig, ag, neut=targets[ag]['neut'], A_adv = True, A_codon = True, A_neut=True, 
                      E=1.0, B=1.0, 
             use_norm=True, use_max=False, use_codon_weights=False, logo=False))

df = pd.concat(df)
plot_res_logo(df, f"../plots/Fig3/_expr-bind-calc-logo", site_thres=0.5, width=6)
df.to_csv(f"../data/calc/_source-data-expr-bind-calc-logo.csv", index=None)
    
pd.concat(df_site).to_csv(f"../data/calc/_site_data-sum-expr-bind.csv", index=None)


185
185
167
167
142
142
394
394
372
372
323
323
[377 378 384 403 405 406 408 417 420 421 427 428 439 440 441 444 445 455
 456 462 465 468 471 472 474 475 476 478 479 482 485 486 487 493 504 505]
[377 378 384 403 405 406 408 417 420 421 427 428 439 440 441 444 445 455
 456 462 465 468 471 472 474 475 476 478 479 482 485 486 487 493 504 505]
['reinfection_cross_JN1 weighting JN1_IC50'
 'reinfection_cross_KP2 weighting JN1_R346T_F456L_IC50'
 'reinfection_cross_KP3 weighting KP3_IC50'
 'reinfection_specific_JN1 weighting JN1_IC50'
 'reinfection_specific_KP2 weighting JN1_R346T_F456L_IC50'
 'reinfection_specific_KP3 weighting KP3_IC50']


In [8]:
df = []
df_site = []

for ag in targets:
    use_abs = targets[ag]['use_abs']
    scores_orig = targets[ag]['dms'].query('antibody in @use_abs')

    df.append(do_calc(scores_orig, ag, neut=targets[ag]['neut'], A_adv = True, A_codon = True, A_neut=True, 
                      E=1.0, B=0.0, 
             use_norm=True, use_max=False, use_codon_weights=False, logo=True, muts=[]).rename(
        columns={'mut_escape_adj':'mut_escape'}).assign(
        target=ag,
        antibody=ag+' weighting '+targets[ag]['neut'],
    ))

    df_site.append(do_calc(scores_orig, ag, neut=targets[ag]['neut'], A_adv = True, A_codon = True, A_neut=True, 
                      E=1.0, B=0.0, 
             use_norm=True, use_max=False, use_codon_weights=False, logo=False))

df = pd.concat(df)
plot_res_logo(df, f"../plots/Fig3/_expr-no-bind-calc-logo", site_thres=0.7, width=6)
df.to_csv(f"../data/calc/_source-data-expr-no-bind-calc-logo.csv", index=None)
    
pd.concat(df_site).to_csv(f"../data/calc/_site_data-sum-expr-no-bind.csv", index=None)


185
185
167
167
142
142
394
394
372
372
323
323
[403 405 439 455 456 472 473 474 475 476 480 485 487 488 489 493 502 504
 505]
[403 405 439 455 456 472 473 474 475 476 480 485 487 488 489 493 502 504
 505]
['reinfection_cross_JN1 weighting JN1_IC50'
 'reinfection_cross_KP2 weighting JN1_R346T_F456L_IC50'
 'reinfection_cross_KP3 weighting KP3_IC50'
 'reinfection_specific_JN1 weighting JN1_IC50'
 'reinfection_specific_KP2 weighting JN1_R346T_F456L_IC50'
 'reinfection_specific_KP3 weighting KP3_IC50']
